In [1]:
from web3 import Web3

In [2]:
# Conexión a un nodo Sepolia (usando Infura en este ejemplo)
sepolia_url = "https://sepolia.infura.io/v3/b1f4149cae3f47c580b3027e0faa7923"  # Reemplaza con tu URL de Infura o un nodo similar
web3 = Web3(Web3.HTTPProvider(sepolia_url))

# Verifica si estás conectado a la red Sepolia
if web3.is_connected():
    print("Conectado a la red Sepolia")
else:
    print("No se pudo conectar a la red Sepolia")

Conectado a la red Sepolia


In [5]:
# Dirección del contrato desplegado en Sepolia
contract_address = "0x33cc0fb2d0d3b0a867d50eb3d465f50efd054ea7"  # Reemplaza con la dirección de tu contrato

In [4]:
# ABI del contrato (copiado desde Remix) - Application Binary Interface -
contract_abi = [
    {
        "inputs": [],
        "stateMutability": "nonpayable",
        "type": "constructor"
    },
    {
        "inputs": [
            {
                "internalType": "uint256",
                "name": "_candidateId",
                "type": "uint256"
            }
        ],
        "name": "getCandidate",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            },
            {
                "internalType": "string",
                "name": "",
                "type": "string"
            },
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "candidatesCount",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "name": "candidates",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "id",
                "type": "uint256"
            },
            {
                "internalType": "string",
                "name": "name",
                "type": "string"
            },
            {
                "internalType": "uint256",
                "name": "voteCount",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "",
                "type": "address"
            }
        ],
        "name": "voters",
        "outputs": [
            {
                "internalType": "bool",
                "name": "",
                "type": "bool"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "uint256",
                "name": "_candidateId",
                "type": "uint256"
            }
        ],
        "name": "vote",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    }
]

In [7]:
# Convertir la dirección al formato checksum
contract_address_checksum = Web3.to_checksum_address(contract_address)

# Ahora utiliza esta dirección en la instancia del contrato
contract = web3.eth.contract(address=contract_address_checksum, abi=contract_abi)

In [9]:
# Ejemplo 1: Consultar información de un candidato
def get_candidate_info(candidate_id):
    try:
        candidate = contract.functions.getCandidate(candidate_id).call()
        print(f"ID: {candidate[0]}, Nombre: {candidate[1]}, Votos: {candidate[2]}")
    except Exception as e:
        print(f"Error al obtener la información del candidato: {e}")

In [10]:
get_candidate_info(1)  # Obtén información del candidato con ID 1

ID: 1, Nombre: Alice, Votos: 0


In [11]:
# Ejemplo 2: Verificar si una dirección ya votó
def has_voted(voter_address):
    try:
        voted = contract.functions.voters(voter_address).call()
        print(f"La dirección {voter_address} ya ha votado: {voted}")
    except Exception as e:
        print(f"Error al verificar si la dirección ha votado: {e}")

In [12]:
has_voted("0x7b17528eE8Bd97F13bC3b5Ad27ccf9Ce43Cdc826")  # Reemplaza con una dirección para verificar si ha votado

La dirección 0x7b17528eE8Bd97F13bC3b5Ad27ccf9Ce43Cdc826 ya ha votado: False


In [8]:
# Ejemplo 3: Emitir un voto
def vote(candidate_id, account_address, private_key):
    try:
        # Configurar la transacción
        transaction = contract.functions.vote(candidate_id).build_transaction({
            'from': account_address,
            'nonce': web3.eth.get_transaction_count(account_address),
            'gas': 2000000,
            'gasPrice': web3.to_wei('50', 'gwei'),
            'chainId': 11155111  # Sepolia Chain ID
        })

        # Firmar la transacción
        signed_txn = web3.eth.account.sign_transaction(transaction, private_key)

        # Enviar la transacción
        txn_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)

        # Esperar a que se mine la transacción
        txn_receipt = web3.eth.wait_for_transaction_receipt(txn_hash)
        print(f"Voto emitido con éxito. Hash de transacción: {txn_receipt.transactionHash.hex()}")
    except Exception as e:
        print(f"Error al emitir el voto: {e}")

In [ ]:
# vote(1, "0x7b17528eE8Bd97F13bC3b5Ad27ccf9Ce43Cdc826", "0xYourPrivateKeyHere")  # Reemplaza con la dirección y clave privada para votar